# pyboj Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/pyboj/blob/main/examples/quickstart.ipynb)

Python client for the [Bank of Japan Time-Series Statistics API](https://www.stat-search.boj.or.jp/).

Based on the official API [announced 2026-02-18](https://www.boj.or.jp/statistics/outline/notice_2026/not260218a.htm).

## Features
- **`BOJ` client** with typed domain methods — no magic strings
- **Enum-driven filtering** for currencies, industries, rate types, etc.
- **Domain wrappers** with parsed dates, typed values, and pandas conversion

In [ ]:
# Install the library
!pip install -q pyboj

In [ ]:
from pyboj import (
    BOJ, Currency, Frequency, Database,
    TankanIndustry, TankanSize, RateCategory, IndexType,
    BopAccount, MonetaryComponent, IndustrySector,
    MarketSegment, AccountSide, InstitutionType,
    FofSector, OperationType, FiscalItem, StatCategory,
)

## 1. Exchange Rates with Currency Enum

In [ ]:
boj = BOJ()

# Fetch USD/JPY daily exchange rates
rates = boj.exchange_rates(
    currency=Currency.USD_JPY,
    frequency=Frequency.D,
    start_date="202401",
    end_date="202403",
)

for r in rates:
    print(f"{r.currency_pair} | {r.rate_type}")
    print(f"  Dates: {r.dates[:3]}")
    print(f"  Values: {r.values[:3]}")

## 2. DataFrame Conversion + Plotting

In [ ]:
# Convert to pandas DataFrame and plot
if rates:
    df = rates[0].to_dataframe()
    print(df.head(10))
    df.plot(title=f"{rates[0].currency_pair} - {rates[0].rate_type.value}", figsize=(10, 4))

## 3. Interest Rates

In [ ]:
# Fetch call rates
ir = boj.interest_rates(
    category=RateCategory.CALL_RATE,
    frequency=Frequency.D,
    start_date="202401",
    end_date="202403",
)

for r in ir[:3]:
    print(f"{r.rate_category} | {r.collateralization} | tenor={r.tenor}")
    print(f"  Values: {r.values[:5]}")

## 4. TANKAN Survey

In [ ]:
# Fetch TANKAN data for large manufacturers
tankan = boj.tankan(
    industry=TankanIndustry.MANUFACTURING,
    size=TankanSize.LARGE,
    start_date="202301",
    end_date="202412",
)

for t in tankan[:5]:
    print(f"{t.industry} | {t.size} | {t.item} | {t.series_type}")
    print(f"  Values: {t.values[:4]}")

## 5. Price Indices

In [ ]:
# Fetch producer price indices
indices = boj.price_indices(
    index_type=IndexType.PRODUCER,
    start_date="202401",
    end_date="202412",
)

for idx in indices[:3]:
    print(f"{idx.index_type} | base={idx.base_year} | yoy={idx.is_yoy_change}")
    print(f"  Values: {idx.values[:4]}")

## 6. Balance of Payments

In [ ]:
# Fetch balance of payments — current account
bop = boj.balance_of_payments(
    account=BopAccount.CURRENT,
    start_date="202401",
    end_date="202412",
)

for b in bop[:3]:
    print(f"{b.account} | {b.name}")
    print(f"  Values: {b.values[:4]}")

## 7. Money & Deposits

In [ ]:
# Fetch monetary base data
money = boj.money_deposits(
    component=MonetaryComponent.TOTAL,
    db=Database.MONETARY_BASE,
    start_date="202401",
    end_date="202412",
)

for m in money[:3]:
    print(f"{m.component} | {m.adjustment} | {m.name}")
    print(f"  Values: {m.values[:4]}")

## 8. Loans

In [ ]:
# Fetch loans by industry sector
loans = boj.loans(
    sector=IndustrySector.MANUFACTURING,
    start_date="202401",
    end_date="202412",
)

for l in loans[:3]:
    print(f"{l.sector} | {l.name}")
    print(f"  Values: {l.values[:4]}")

## 9. Financial Markets

In [ ]:
# Fetch government bond trading data
fm = boj.financial_markets(
    segment=MarketSegment.GOVT_BONDS,
    db=Database.GOVT_BOND_TRADING,
    start_date="202401",
    end_date="202412",
)

for f in fm[:3]:
    print(f"{f.segment} | {f.instrument_type} | {f.name}")
    print(f"  Values: {f.values[:4]}")

## 10. Balance Sheets

In [ ]:
# Fetch BOJ balance sheet — assets side
bs = boj.balance_sheets(
    account_side=AccountSide.ASSETS,
    institution_type=InstitutionType.BOJ,
    start_date="202401",
    end_date="202412",
)

for b in bs[:3]:
    print(f"{b.account_side} | {b.institution_type} | {b.name}")
    print(f"  Values: {b.values[:4]}")

## 11. Flow of Funds

In [ ]:
# Fetch household flow of funds
fof = boj.flow_of_funds(
    sector=FofSector.HOUSEHOLDS,
    start_date="202301",
    end_date="202412",
)

for f in fof[:3]:
    print(f"{f.sector} | {f.instrument} | {f.name}")
    print(f"  Values: {f.values[:4]}")

## 12. BOJ Operations

In [ ]:
# Fetch JGB operations
ops = boj.boj_operations(
    operation_type=OperationType.JGB_OPERATIONS,
    start_date="202401",
    end_date="202412",
)

for o in ops[:3]:
    print(f"{o.operation_type} | {o.name}")
    print(f"  Values: {o.values[:4]}")

## 13. Public Finance

In [ ]:
# Fetch tax revenue data
pf = boj.public_finance(
    fiscal_item=FiscalItem.TAX_REVENUE,
    start_date="202401",
    end_date="202412",
)

for p in pf[:3]:
    print(f"{p.fiscal_item} | {p.name}")
    print(f"  Values: {p.values[:4]}")

## 14. International Statistics

In [ ]:
# Fetch derivatives market statistics
intl = boj.international(
    stat_category=StatCategory.DERIVATIVES,
    db=Database.DERIVATIVES_MARKET,
    start_date="202301",
    end_date="202412",
)

for i in intl[:3]:
    print(f"{i.stat_category} | {i.name}")
    print(f"  Values: {i.values[:4]}")

## 15. Layer Tree & Search (Discovery)

In [ ]:
# Build a hierarchical layer tree for exchange rates
tree = boj.layer_tree(Database.EXCHANGE_RATES)
print(f"Root has {len(tree.children)} top-level categories:\n")
for child in tree.children:
    print(f"  {child.name} ({len(child.series_codes)} series, {len(child.children)} subcategories)")

In [ ]:
# Search metadata by keyword
results = boj.search(Database.EXCHANGE_RATES, "USD")
print(f"Found {len(results)} series matching 'USD':\n")
for rec in results[:5]:
    print(f"  {rec.SERIES_CODE} | {rec.NAME_OF_TIME_SERIES}")

# Clean up
boj.close()